In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
pwd

'/Users/uw-user/Library/CloudStorage/GoogleDrive-vvithurshan@gmail.com/My Drive/Artificial-Intelligence-2025/Machine-Learning/PW-Skills/Projects/Chiken-Disease-Classification'

In [4]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
# Configurationmanager
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ]) 

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config

        
    

In [8]:
# Component
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [13]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_ckpt_callbacks,
            self._create_ckpt_callbacks
        ]

In [14]:
# pipeline
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2025-07-18 19:14:44,530: INFO: common: YAML file loaded successfully: config/config.yaml]
[2025-07-18 19:14:44,533: INFO: common: YAML file loaded successfully: params.yaml]
[2025-07-18 19:14:44,533: INFO: common: Created Directory at: artifacts]
[2025-07-18 19:14:44,534: INFO: common: Created Directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2025-07-18 19:14:44,534: INFO: common: Created Directory at: artifacts/prepare_callbacks/tensorboard_log_dir]


In [15]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2025-07-18-19-16-48'